Попробуем свои силы в соревновании по распозхнованию рукописные цифры из набора данных MNIST на сайте <a href="https://www.kaggle.com/c/digit-recognizer">kaggle.com</a>

In [5]:
import numpy as np
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
import sys
import os.path

Using TensorFlow backend.


In [6]:
np.random.seed(42)

In [10]:
img_rows, img_cols = 28, 28

## Загрузка данных для обучения

In [7]:
train_dataset = np.loadtxt('Digit Recognizer/train.csv', skiprows=1, dtype='int', delimiter=',')

Выделяем данные для обучени

In [8]:
x_train = train_dataset[:, 1:]

Переформатируем данные в 2D, бэкенд Tensorflow

In [11]:
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

Нормализуем данные

In [12]:
x_train = x_train.astype("float32")
x_train /= 255.0

Выделяем правильные ответы

In [13]:
y_train = train_dataset[:, 0]

Преобразуем правильные ответы в категоризированное представление

In [14]:
y_train = np_utils.to_categorical(y_train)

Загружаем данные для предсказания

In [16]:
test_dataset = np.loadtxt('Digit Recognizer/test.csv', skiprows=1, delimiter=",")

Переформатируем данные в 2D, бэкенд TensorFlow

In [17]:
x_test = test_dataset.reshape(test_dataset.shape[0], img_rows, img_cols, 1)
x_test /= 255.0

## Создание и обучение нейронной сети

Создаем последовательную модель

In [18]:
model = Sequential()

In [19]:
model.add(Conv2D(75, kernel_size=(5, 5),
                 activation='relu',
                 input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(100, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(500, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

Компилируем модель

In [20]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [21]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 24, 24, 75)        1950      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 75)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 75)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 100)         187600    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 4, 100)         0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 4, 4, 100)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1600)              0         
__________

Обучаем сеть

In [22]:
model.fit(x_train, y_train, batch_size=200, epochs=10, verbose=2)

Epoch 1/10
 - 418s - loss: 0.2759 - acc: 0.9130
Epoch 2/10
 - 374s - loss: 0.0718 - acc: 0.9784
Epoch 3/10
 - 412s - loss: 0.0506 - acc: 0.9846
Epoch 4/10
 - 394s - loss: 0.0399 - acc: 0.9874
Epoch 5/10
 - 429s - loss: 0.0348 - acc: 0.9888
Epoch 6/10
 - 464s - loss: 0.0279 - acc: 0.9912
Epoch 7/10
 - 441s - loss: 0.0232 - acc: 0.9929
Epoch 8/10
 - 515s - loss: 0.0198 - acc: 0.9932
Epoch 9/10
 - 463s - loss: 0.0204 - acc: 0.9931
Epoch 10/10
 - 473s - loss: 0.0195 - acc: 0.9936


Распознаем цифры на изображениях

In [23]:
predictions = model.predict(x_test)

Преобразуем предсказания из категориального представления в метки классов

In [24]:
predictions = np.argmax(predictions, axis=1)

Записываем результаты распознавания в текстовый файл

In [25]:
out = np.column_stack((range(1, predictions.shape[0]+1), predictions))
np.savetxt('submission_Digit Recognizer.csv', out, header="ImageId,Label", 
            comments="", fmt="%d,%d")